In [1]:
import math

In [2]:
import pandas as pd
from pandas.api.types import is_bool_dtype
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [3]:
import numpy as np

In [4]:
class Node(object):
    def __init__(self, value, neighbours=[]):
        self.value = value
        self.neighbours = neighbours
        
    def get_value(self):
        return self.value
    
    def get_neighbours(self):
        return self.neighbours

In [5]:
class TreeNode(Node):
    def __init__(self, value, children=[], parent=None):
        Node.__init__(self, value=value, neighbours=None)
        
        self.children = children
        self.parent = parent
        
    def get_children(self):
        return self.children
        
    def get_parent(self):
        return self.parent
    
    def is_root(self):
        return self.parent == None
    
    def is_leaf(self):
        return self.children == []
    
    def get_depth(self):
        if self.is_root():
            return 0
        else:
            return 1 + self.parent.get_depth()

In [6]:
class Leaf(TreeNode):
    def __init__(self, value, parent):
        TreeNode.__init__(self, value=value, children=[], parent=parent)

In [7]:
class DecisionNode(TreeNode):
    def __init__(self, argument, value=None, right=None, left=None, parent=None):
        TreeNode.__init__(self, value=argument, children=[left, right], parent=parent)
        self.argument_value = value
        self.argument = argument
    
    def get_argument_value(self):
        return self.argument_value
    
    def get_argument(self):
        return self.value
    
    def get_side(self):
        if self.is_root():
            return None
        else:
            if self == self.parent.get_children()[0]:
                return 'left'
            else:
                return 'right'

In [8]:
def random_sampling(k, data=None):
    if data == None:
        def sampling(data):
            m = k
            if k > data.shape[0]:
                m = data.shape[0]
            return data.sample(n=m)
        return sampling
    else:
        return data.sample(n=k)

In [9]:
class DecisionTree(object):
    def __init__(self, max_depth=2, result_label='label', numerical_sampler=random_sampling(5)):
        self.arguments = None
        self.max_depth = max_depth
        self.tree = None
        self.result_label = result_label
        self.arg_property = dict()
        self.numerical_sampler = numerical_sampler
        
    def compute_gini_index(self, data, argument, value, result_label='label'):
        left_data  = self._filter(data, argument, value, 'left')
        right_data = self._filter(data, argument, value, 'right')
        
        left_N = left_data.shape[0]
        left_n = left_data[left_data[result_label] == True].shape[0]
        
        try:
            left_gini = 1 - (left_n/left_N) ** 2 - ( (left_N-left_n)/left_N) ** 2
        except:
            left_gini = 1

        right_N = right_data.shape[0]
        right_n = right_data[right_data[result_label] == True].shape[0]
        
        try:
            right_gini = 1 - (right_n/right_N) ** 2 - ( (right_N-right_n)/right_N) ** 2
        except:
            right_gini = 1

        try:
            return left_N/(left_N+right_N) * left_gini + right_N/(left_N+right_N)*right_gini
        except:
            return 1
    
    def get_best_split(self, data, arguments):
        min_arg = None
        min_E   = 1
        
        for arg in arguments:
            if self.arg_property[arg]['type'] == 'categorical':
                for value in self.arg_property[arg]['classes']:
                    E = self.compute_gini_index(data, arg, value, self.result_label)
                    if E < min_E:
                        min_E = E
                        min_arg = arg
                        min_val = value
            elif self.arg_property[arg]['type'] == 'numerical':
                for value in self.numerical_sampler(data[arg]).values:
                    E = self.compute_gini_index(data, arg, value, self.result_label)
                    if E < min_E:
                        min_E = E
                        min_arg = arg
                        min_val = value
                         
        return min_arg, min_val, min_E 
    
    def _filter(self, data, argument, val, side):
        if self.arg_property[argument]['type'] == 'categorical':
            if side == 'left':
                return data[data[argument] != val]
            else:
                return data[data[argument] == val]
        elif self.arg_property[argument]['type'] == 'numerical':
            if side == 'left':
                return data[data[argument] < val]
            else:
                return data[data[argument] >= val]
        else:
            return None
        
    def _check(self, node, value):
        if self.arg_property[node.get_argument()]['type'] == 'categorical':
            if node.get_argument_value() == value:
                return 'right'
            else:
                return 'left'
        elif self.arg_property[node.get_argument()]['type'] == 'numerical':
            if value < node.get_argument_value():
                return 'left'
            else:
                return 'right'
        else:
            return None
    
    def train(self, data, arguments):
        self.arguments = arguments
        self._compute_arguments_properties(data)
        
        self.tree = self._build_tree(data, arguments=set(self.arguments), parent=None, level=0)
        
    def _build_tree(self, data, arguments, parent=None, level=0):
        if len(arguments) != 0 and level < self.max_depth:
            min_arg, min_val, min_gini = self.get_best_split(data, list(arguments))
            node = DecisionNode(argument=min_arg, value=min_val, parent=parent)
            
            data_left = self._filter(data, min_arg, min_val, 'left')
            data_right  = self._filter(data, min_arg, min_val, 'right')
            
            arguments = arguments - { min_arg }

            if data_left.shape[0] != 0 and data_right.shape[0] != 0:
                node.left  = self._build_tree(data_left, arguments=arguments, parent=node, level=level+1)
                node.right = self._build_tree(data_right,  arguments=arguments, parent=node, level=level+1)

                return node

        N = data.shape[0]
        n = data[data[self.result_label] == True].shape[0]

        node = Leaf({'True' : n/N, 'False' : (N-n)/N}, parent=parent)
        return node
    
    def _compute_arguments_properties(self, data):
        self.arg_property = dict()
        
        for argument in self.arguments:
            if is_string_dtype(data[argument]) or is_bool_dtype(data[argument]):
                self.arg_property[argument] = { 'type' : 'categorical' , 'classes' : data[argument].unique() }
            else:
                self.arg_property[argument] = { 'type' : 'numerical' }
                
    def predict(self, data):
        node = self.tree
        
        while not node.is_leaf():
            side = self._check(node, data[node.get_argument()])
            if side == 'left':
                node = node.left
            elif side == 'right':
                node = node.right
            else:
                return None
                
        return node.get_value()

In [10]:
def print_tree(node):
    print(node.value)
    
    if isinstance(node, Leaf):
        return
    else:
        if node.left != None:
            print('left: ')
            print_tree(node.left)
        else:
            pass

        if node.right != None:
            print('right: ')
            print_tree(node.right)
        else:
            pass

### Test - XOR

In [11]:
DC = DecisionTree()
data = pd.DataFrame({
    'gender': [True, True, False, False],
    'rainy': [True, False, True, False],
    'label': [False, True, True, False]
})

In [12]:
DC.train(data, ['rainy', 'gender'])

In [13]:
print( DC.predict({'gender': True, 'rainy': True }) )
print( DC.predict({'gender': False, 'rainy': False }) )
print( DC.predict({'gender': True, 'rainy': False }) )
print( DC.predict({'gender': False, 'rainy': True }) )

{'False': 1.0, 'True': 0.0}
{'False': 1.0, 'True': 0.0}
{'False': 0.0, 'True': 1.0}
{'False': 0.0, 'True': 1.0}


### Test - Tic-Tac-Toe dataset

In [14]:
tic_tac_toe_data = pd.read_csv('../datasets/tic-tac-toe.csv')

In [15]:
DC2 = DecisionTree(max_depth=20)
DC2.train(tic_tac_toe_data, tic_tac_toe_data.columns[:-1].values)

In [16]:
for nth in range(tic_tac_toe_data.shape[0]):
    print('label: ', tic_tac_toe_data.iloc[nth][-1], ' prediction: ', DC2.predict(dict(tic_tac_toe_data.iloc[nth])) )

label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  Tr

label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  Tr

label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  Tr

label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 0.5, 'True': 0.5}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  False  prediction:  {'False': 1.0, 'True

### Test - Hepatitis dataset

In [23]:
hepatitis_data = pd.read_csv('../datasets/hepatitis.csv', header=None)
hepatitis_data = hepatitis_data.rename(columns={0: 'label'})

In [24]:
#transform label
for index, row in hepatitis_data.iterrows():
    if row['label'] == 1:
        hepatitis_data.loc[index, 'label'] = False
    else:
        hepatitis_data.loc[index, 'label'] = True

In [25]:
DC3 = DecisionTree(max_depth=20)
DC3.train(hepatitis_data, hepatitis_data.columns[:-1].values)

In [26]:
for nth in range(hepatitis_data.shape[0]):
    print('label: ', hepatitis_data.iloc[nth]['label'], ' prediction: ', DC3.predict(dict(hepatitis_data.iloc[nth])) )

label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  False  prediction:  {'False': 1.0, 'True': 0.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  True  prediction:  {'False': 0.0, 'True': 1.0}
label:  T